In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.metrics import mean_squared_error

In [13]:
white_wine = pd.read_csv('https://raw.githubusercontent.com/chemo-wakate/tutorial-6th/master/beginner/data/winequality-white.txt', sep='\t', index_col=0) 
white_wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [14]:
red_wine = pd.read_csv('https://raw.githubusercontent.com/chemo-wakate/tutorial-6th/master/beginner/data/winequality-red.txt', sep='\t', index_col=0) 
red_wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [15]:
random_state = 0
X_train, X_test, y_train, y_test = train_test_split(red_wine.iloc[:, :-1].values, red_wine.iloc[:, [-1]].values, test_size=.3, random_state=random_state) 
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=.3, random_state=random_state) 

In [16]:
print(type(X_train))
print(type(X_test))
print(type(y_train))
print(type(y_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [17]:
SEED = 53
N_STEPS = 3
tabnet_params = dict(n_d=8, n_a=8, n_steps=N_STEPS, gamma=1.3,
                     n_independent=2, n_shared=2,
                     seed=SEED, lambda_sparse=1e-3, 
                     optimizer_fn=torch.optim.Adam, 
                     optimizer_params=dict(lr=2e-2),
                     mask_type="entmax",
                     scheduler_params=dict(mode="min",
                                           patience=5,
                                           min_lr=1e-5,
                                           factor=0.9,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=10
                    )

model = TabNetRegressor(**tabnet_params)
# model = TabNetClassifier() 
model.fit(
  X_train, y_train,
  eval_set=[(X_valid, y_valid)],
  max_epochs=5000,
  patience=100,
  eval_metric={'rmse'} # 'rmsle', 'mse', 'mae'
  # from_unsupervised=pretrainer
)

Device used : cpu
epoch 0  | loss: 32.64986| val_0_rmse: 11.02902|  0:00:00s
epoch 10 | loss: 3.95385 | val_0_rmse: 10.26458|  0:00:01s
epoch 20 | loss: 0.89716 | val_0_rmse: 2.05911 |  0:00:03s
epoch 30 | loss: 0.54875 | val_0_rmse: 2.106   |  0:00:04s
epoch 40 | loss: 0.45242 | val_0_rmse: 2.04415 |  0:00:05s
epoch 50 | loss: 0.39261 | val_0_rmse: 1.96964 |  0:00:07s
epoch 60 | loss: 0.37867 | val_0_rmse: 2.37715 |  0:00:08s
epoch 70 | loss: 0.36645 | val_0_rmse: 2.61273 |  0:00:09s
epoch 80 | loss: 0.36892 | val_0_rmse: 2.4656  |  0:00:10s
epoch 90 | loss: 0.35189 | val_0_rmse: 2.21405 |  0:00:11s
epoch 100| loss: 0.33867 | val_0_rmse: 1.92403 |  0:00:13s
epoch 110| loss: 0.32406 | val_0_rmse: 1.75426 |  0:00:15s
epoch 120| loss: 0.32064 | val_0_rmse: 1.64614 |  0:00:16s

Early stopping occurred at epoch 123 with best_epoch = 23 and best_val_0_rmse = 1.4265
Best weights from best epoch are automatically used!


In [18]:
test_score = mean_squared_error(model.predict(X_test), y_test)

print(f"BEST VALID SCORE: {model.best_cost}")
print(f"FINAL TEST SCORE: {test_score}")

BEST VALID SCORE: 1.426498589791503
FINAL TEST SCORE: 2.1107192499291902


In [19]:
X_test

array([[10.8 ,  0.47,  0.43, ...,  3.17,  0.76, 10.8 ],
       [ 8.1 ,  0.82,  0.  , ...,  3.36,  0.53,  9.6 ],
       [ 9.1 ,  0.29,  0.33, ...,  3.26,  0.84, 11.7 ],
       ...,
       [ 7.8 ,  0.55,  0.35, ...,  3.25,  0.56,  9.2 ],
       [ 6.8 ,  0.65,  0.02, ...,  3.35,  0.62, 10.4 ],
       [ 8.  ,  0.43,  0.36, ...,  3.34,  0.46,  9.4 ]])

In [21]:
X_test

array([[10.8 ,  0.47,  0.43, ...,  3.17,  0.76, 10.8 ],
       [ 8.1 ,  0.82,  0.  , ...,  3.36,  0.53,  9.6 ],
       [ 9.1 ,  0.29,  0.33, ...,  3.26,  0.84, 11.7 ],
       ...,
       [ 7.8 ,  0.55,  0.35, ...,  3.25,  0.56,  9.2 ],
       [ 6.8 ,  0.65,  0.02, ...,  3.35,  0.62, 10.4 ],
       [ 8.  ,  0.43,  0.36, ...,  3.34,  0.46,  9.4 ]])

In [22]:
print(X_test.shape)

(480, 11)


In [ ]:
## One-Hot encodingの練習